In [6]:

import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")
from defs import my_dateparser, depois_do_feriado


In [2]:
np.random.seed(1234)

## Pré-processamento

In [32]:
df0 = pd.read_pickle('data/raw/query_result.pickle')
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140321 entries, 0 to 140320
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   cd_estabelecimento     140321 non-null  int64         
 1   nr_atendimento         140321 non-null  int64         
 2   dt_entrada             140321 non-null  datetime64[ns]
 3   dt_alta                139831 non-null  datetime64[ns]
 4   ds_tipo_atendimento    140321 non-null  object        
 5   ds_carater_internacao  140249 non-null  object        
 6   dt_entrada_unidade     140321 non-null  datetime64[ns]
 7   dt_saida_unidade       140054 non-null  datetime64[ns]
 8   ds_classific_setor     140321 non-null  object        
dtypes: datetime64[ns](4), int64(2), object(3)
memory usage: 9.6+ MB


In [35]:
df0['dia_entrada_unidade'] = pd.to_datetime(df0['dt_entrada_unidade'].dt.date)
df0['dia_saida_unidade'] = pd.to_datetime(df0['dt_saida_unidade'].dt.date)

In [52]:
from tqdm.notebook import tqdm
internados_ui = []
internados_uti = []
days = df0['dia_entrada_unidade'].unique()
for day in tqdm(sorted(days)):
    df_ui = df0[(df0.dt_entrada_unidade < day) &
                (df0.ds_classific_setor == 'Unidades de Internação') &
                ((df0.dt_saida_unidade > day) | (df0.dt_saida_unidade.isna() == True))].drop_duplicates(subset=['nr_atendimento'])
    df_uti = df0[(df0.dt_entrada_unidade < day) &
                (df0.ds_classific_setor == 'UTI') &
                ((df0.dt_saida_unidade > day) | (df0.dt_saida_unidade.isna() == True))].drop_duplicates(subset=['nr_atendimento'])
    
    internados_ui.append({'ds':day,'y':len(df_ui)})
    internados_uti.append({'ds':day,'y':len(df_uti)})

  0%|          | 0/1891 [00:00<?, ?it/s]

,ds,y
0,2017-01-01,0
1,2017-01-02,15
2,2017-01-03,56
3,2017-01-04,103
4,2017-01-05,134
...,...,...
1886,2022-03-02,211
1887,2022-03-03,224
1888,2022-03-04,255
1889,2022-03-05,229


In [4]:
df['DT_ENTRADA_UNIDADE'] = df['DT_ENTRADA_UNIDADE'].apply(my_dateparser)
df['DT_SAIDA_UNIDADE'] = df['DT_SAIDA_UNIDADE'].apply(my_dateparser)

In [5]:
df['DIA_ENTRADA_UNIDADE'] = pd.to_datetime(df['DT_ENTRADA_UNIDADE'].dt.date)
df['DIA_SAIDA_UNIDADE'] = pd.to_datetime(df['DT_SAIDA_UNIDADE'].dt.date)

In [6]:
df_uti = df[df['DS_CLASSIFIC_SETOR'] == 'UTI'].copy()
df_ui = df[df['DS_CLASSIFIC_SETOR'] == 'Unidades de Internação'].copy()

del df

In [7]:
df_entrada_uti = df_uti[['DIA_ENTRADA_UNIDADE', 'NR_ATENDIMENTO']].groupby('DIA_ENTRADA_UNIDADE').count().reset_index()
df_saida_uti = df_uti[['DIA_SAIDA_UNIDADE', 'NR_ATENDIMENTO']].groupby('DIA_SAIDA_UNIDADE').count().reset_index()

df_entrada_ui = df_ui[['DIA_ENTRADA_UNIDADE', 'NR_ATENDIMENTO']].groupby('DIA_ENTRADA_UNIDADE').count().reset_index()
df_saida_ui = df_ui[['DIA_SAIDA_UNIDADE', 'NR_ATENDIMENTO']].groupby('DIA_SAIDA_UNIDADE').count().reset_index()

del df_uti
del df_ui

In [8]:
df_entrada_uti.columns = ["ds", "y"]
df_saida_uti.columns = ["ds", "y"]
df_entrada_ui.columns = ["ds", "y"]
df_saida_ui.columns = ["ds", "y"]

In [9]:
df_entrada_uti.to_pickle('data/interim/entrada_uti.pickle')
df_saida_uti.to_pickle('data/interim/saida_uti.pickle')
df_entrada_ui.to_pickle('data/interim/entrada_ui.pickle')
df_saida_ui.to_pickle('data/interim/saida_ui.pickle')

In [10]:
dateparse = lambda dates: [datetime.strptime(d, '%Y-%m-%d') for d in dates]
holidays = pd.read_csv('data/raw/holidays.csv', parse_dates=['0'], date_parser=dateparse)
holidays.columns = ['ds']

after_holidays = holidays.copy()
after_holidays['ds'] = holidays.apply(depois_do_feriado, axis=1)
after_holidays.dropna(inplace=True)
after_holidays['holiday'] = 1
after_holidays.rename(columns={'holiday':'after_holidays'}, inplace=True)

In [11]:
after_holidays = df_entrada_uti.merge(after_holidays, left_on="ds", right_on="ds", how="left").fillna(0)[['ds', 'after_holidays']]
after_holidays = after_holidays[after_holidays['ds'].duplicated() == False]

In [12]:
after_holidays

,ds,after_holidays
0,2017-01-01,0.0
1,2017-01-02,0.0
2,2017-01-03,0.0
3,2017-01-04,0.0
4,2017-01-05,0.0
...,...,...
1882,2022-02-27,0.0
1883,2022-02-28,0.0
1884,2022-03-01,1.0
1885,2022-03-02,1.0


In [13]:
fim_ano = df_entrada_uti.copy()
fim_ano.loc[(fim_ano['ds'].dt.month == 12) & (fim_ano['ds'].dt.day >= 15), 'fim_ano'] = 1
fim_ano.loc[(fim_ano['ds'].dt.month == 1) & (fim_ano['ds'].dt.day >= 10), 'fim_ano'] = 1
fim_ano['fim_ano'].fillna(0, inplace=True)
fim_ano.drop('y', axis=1, inplace=True)

In [14]:
fim_ano.to_pickle('data/interim/fim_ano.pickle')
after_holidays.to_pickle('data/interim/after_holidays.pickle')